# Consolidating training data

Here we are going to consolidate a csv file with a dataframe containing all the data we need for fine-tuning BERT. For the preliminary results, we are not going to pre-train BERT with the domain-specific language corpus. We are going to use the "vanilla" BERT pre-trained on WordPiece. 

Note: the time interval is still just a day, but that is not particularly relevant for the next deadline. 

## IT IS ON BABEYYY

In [1]:
%run functions.py

In [2]:
base_url = 'https://www.sec.gov{}'

In [107]:
filings = []
num_days = 5

for i in range(num_days):
    print('Scraping page', i)
    forms = get_current_events(i, '424B5')
    for link in forms['link_0']:
        docs = get_filing_documents(base_url.format(link))
        doc_link = docs.loc[docs.Type == '424B5', 'Document'].values[0]
        download_sec_documents(doc_link)
        

forms.insert(6, 'Filing', np.nan)
forms

Scraping page 0
Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4


,Date Filed,Form,CIK Code,Company Name,link_0,link_1,Filing
0,02-08-2021,424B5,1625072,APT Florida LLC,/Archives/edgar/data/1625072/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1625072,NaN
1,02-08-2021,424B5,1507585,APT Intermediate Holdco LLC,/Archives/edgar/data/1507585/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1507585,NaN
2,02-08-2021,424B5,1625235,APT New Intermediate Holdco LLC,/Archives/edgar/data/1625235/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1625235,NaN
3,02-08-2021,424B5,1625081,APT Pennsylvania LLC,/Archives/edgar/data/1625081/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1625081,NaN
4,02-08-2021,424B5,1507601,APT Sunshine State LLC,/Archives/edgar/data/1507601/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1507601,NaN
...,...,...,...,...,...,...,...
236,02-08-2021,424B5,1625554,Utica Marcellus Texas Pipeline LLC,/Archives/edgar/data/1625554/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1625554,NaN
237,02-08-2021,424B5,1565228,"Vislink Technologies, Inc.",/Archives/edgar/data/1565228/0001493152-21-002...,browse-edgar?action=getcompany&CIK=1565228,NaN
238,02-08-2021,424B5,1625556,Western Plant Services LLC,/Archives/edgar/data/1625556/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1625556,NaN
239,02-08-2021,424B5,1454702,"Wyoming Interstate Company, L.L.C.",/Archives/edgar/data/1454702/0001047469-21-000...,browse-edgar?action=getcompany&CIK=1454702,NaN


In [3]:
files = []
for file in glob('-archives*.htm'):
    files.append(file)
files

['-archives-edgar-data-1549084-000110465921014298-tm214998-1_424b5.htm',
 '-archives-edgar-data-1422892-000121390021008067-ea135095-424b5_sinoglobal.htm',
 '-archives-edgar-data-1630805-000110465921021077-tm214528-5_424b5.htm',
 '-archives-edgar-data-1596812-000121390021008117-ea135162-424b5_enlivex.htm',
 '-archives-edgar-data-1163302-000110465921017073-tm214549-8_424b5.htm',
 '-archives-edgar-data-1696396-000119312521037392-d122077d424b5.htm',
 '-archives-edgar-data-1411688-000110465921013269-tm214599-1_424b5.htm',
 '-archives-edgar-data-1195737-000119312521035723-d45391d424b5.htm',
 '-archives-edgar-data-25743-000119312521036460-d121779d424b5.htm',
 '-archives-edgar-data-1730463-000119312521032531-d49362d424b5.htm',
 '-archives-edgar-data-877890-000119312521036390-d92207d424b5.htm',
 '-archives-edgar-data-1730463-000119312521038752-d49362d424b5.htm',
 '-archives-edgar-data-1579823-000149315221003422-form424b5.htm',
 '-archives-edgar-data-1626971-000110465921021309-tm214090d2_424b5.h

In [7]:
data = []
codes = []
cols = ['CIK Code', 'Filing']
for file in files:
    split = file.split('-')
    CIK = split[4]
    codes.append(CIK)
    content = read_html(file)
    cleaned_content = clean_html(content)
    try:
        paragraphs = get_risk_info(get_header(cleaned_content), content)
        data.append(cleanhtml(paragraphs))
    except:
        data.append(np.nan)

/Users/danielbejarano/functions.py:86: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 86 of the file /Users/danielbejarano/functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  if not soup.find('p'):
/Users/danielbejarano/functions.py:98: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 98 of the file /Users/danielbejarano/functions.py. To get rid of this wa

In [8]:
date = '2021-02-11'
dates = [date] * 138
dates

['2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-11',
 '2021-02-

In [9]:
df = pd.DataFrame({'CIK Code': codes, 'Filing': data, 'Date Filed': dates})
df

,CIK Code,Filing,Date Filed
0,1549084,NaN,2021-02-11
1,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11
2,1630805,An investment inthe Notes involves significant...,2021-02-11
3,1596812,Investingin our securities involves a high deg...,2021-02-11
4,1163302,NaN,2021-02-11
...,...,...,...
133,1684144,An investment in our securitiesinvolves a high...,2021-02-11
134,1582581,Investing in our securities involves significa...,2021-02-11
135,714256,Investingin our common stock involves a high d...,2021-02-11
136,1719714,"Our business, and an investment in the securit...",2021-02-11


In [10]:
df['Date Filed'] = pd.to_datetime(df['Date Filed'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CIK Code    138 non-null    object        
 1   Filing      111 non-null    object        
 2   Date Filed  138 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.4+ KB


In [11]:
df.dropna(axis = 0, how = 'any', inplace = True)
df

,CIK Code,Filing,Date Filed
1,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11
2,1630805,An investment inthe Notes involves significant...,2021-02-11
3,1596812,Investingin our securities involves a high deg...,2021-02-11
5,1696396,Investing in our ADSs involves a high degree o...,2021-02-11
7,1195737,"Investing in the Securities involves risk, inc...",2021-02-11
...,...,...,...
133,1684144,An investment in our securitiesinvolves a high...,2021-02-11
134,1582581,Investing in our securities involves significa...,2021-02-11
135,714256,Investingin our common stock involves a high d...,2021-02-11
136,1719714,"Our business, and an investment in the securit...",2021-02-11


In [14]:
tickers = cach.get(base_url.format('/files/company_tickers.json')).json()
tickers = pd.DataFrame(tickers)
tickers

,0,1,2,3,4,5,6,7,8,9,...,11088,11089,11090,11091,11092,11093,11094,11095,11096,11097
cik_str,320193,789019,1018724,1652044,1293451,1318605,1326801,1577552,1046179,1067983,...,1832352,1831992,1832351,1832250,1831964,1831937,1831868,1831928,1831359,1831270
ticker,AAPL,MSFT,AMZN,GOOG,TCEHY,TSLA,FB,BABA,TSM,BRK-A,...,CAHCU,LMACU,ENNVU,GNACU,NRACU,CLRMU,LMAOU,DLCAU,JWSM-UN,BITE-UN
title,Apple Inc.,MICROSOFT CORP,AMAZON COM INC,Alphabet Inc.,Tencent Holdings Ltd,"Tesla, Inc.",Facebook Inc,Alibaba Group Holding Ltd,TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD,BERKSHIRE HATHAWAY INC,...,CA Healthcare Acquisition Corp.,Liberty Media Acquisition Corp,ECP Environmental Growth Opportunities Corp.,Group Nine Acquisition Corp.,Noble Rock Acquisition Corp,Clarim Acquisition Corp.,LMF Acquisition Opportunities Inc,Deep Lake Capital Acquisition Corp.,Jaws Mustang Acquisition Corp,Bite Acquisition Corp.


In [15]:
tickers = tickers.transpose()
tickers.head()

,cik_str,ticker,title
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1018724,AMZN,AMAZON COM INC
3,1652044,GOOG,Alphabet Inc.
4,1293451,TCEHY,Tencent Holdings Ltd


In [16]:
tickers.columns = ['CIK Code', 'Ticker', 'CompanyName']
df['CIK Code'] = df['CIK Code'].astype(int)
tickers['CIK Code'] = tickers['CIK Code'].astype(int)
tickers.head()

,CIK Code,Ticker,CompanyName
0,320193,AAPL,Apple Inc.
1,789019,MSFT,MICROSOFT CORP
2,1018724,AMZN,AMAZON COM INC
3,1652044,GOOG,Alphabet Inc.
4,1293451,TCEHY,Tencent Holdings Ltd


In [17]:
tickers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11098 entries, 0 to 11097
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CIK Code     11098 non-null  int64 
 1   Ticker       11098 non-null  object
 2   CompanyName  11098 non-null  object
dtypes: int64(1), object(2)
memory usage: 666.8+ KB


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 1 to 137
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CIK Code    111 non-null    int64         
 1   Filing      111 non-null    object        
 2   Date Filed  111 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.5+ KB


In [19]:
train = df.merge(tickers, on = 'CIK Code', how = 'left')
train

,CIK Code,Filing,Date Filed,Ticker,CompanyName
0,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11,SINO,"Sino-Global Shipping America, Ltd."
1,1630805,An investment inthe Notes involves significant...,2021-02-11,BW,"Babcock & Wilcox Enterprises, Inc."
2,1596812,Investingin our securities involves a high deg...,2021-02-11,ENLV,Enlivex Therapeutics Ltd.
3,1696396,Investing in our ADSs involves a high degree o...,2021-02-11,MITO,Stealth BioTherapeutics Corp
4,1195737,"Investing in the Securities involves risk, inc...",2021-02-11,NEA,Nuveen AMT-Free Quality Municipal Income Fund
...,...,...,...,...,...
114,1684144,An investment in our securitiesinvolves a high...,2021-02-11,ZOM,Zomedica Corp.
115,1582581,Investing in our securities involves significa...,2021-02-11,VJET,voxeljet AG
116,714256,Investingin our common stock involves a high d...,2021-02-11,SMTI,Sanara MedTech Inc.
117,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc


In [20]:
train['Date Filed'] = train['Date Filed'].dt.strftime('%Y-%m-%d')
train

,CIK Code,Filing,Date Filed,Ticker,CompanyName
0,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11,SINO,"Sino-Global Shipping America, Ltd."
1,1630805,An investment inthe Notes involves significant...,2021-02-11,BW,"Babcock & Wilcox Enterprises, Inc."
2,1596812,Investingin our securities involves a high deg...,2021-02-11,ENLV,Enlivex Therapeutics Ltd.
3,1696396,Investing in our ADSs involves a high degree o...,2021-02-11,MITO,Stealth BioTherapeutics Corp
4,1195737,"Investing in the Securities involves risk, inc...",2021-02-11,NEA,Nuveen AMT-Free Quality Municipal Income Fund
...,...,...,...,...,...
114,1684144,An investment in our securitiesinvolves a high...,2021-02-11,ZOM,Zomedica Corp.
115,1582581,Investing in our securities involves significa...,2021-02-11,VJET,voxeljet AG
116,714256,Investingin our common stock involves a high d...,2021-02-11,SMTI,Sanara MedTech Inc.
117,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc


In [25]:
ticker_list = list(train['Ticker'])
dates = list(train['Date Filed'])
deltas = [0,7]

In [26]:
tickers_cleaned = [i for i in ticker_list if (str(i) != 'nan' and len(str(i))<5)]
dates_cleaned = train['Date Filed'][:-(len(ticker_list)-len(tickers_cleaned))]

In [27]:
prices = delta_days_ago_and_current(tickers_cleaned, dates_cleaned)
prices.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,Reference Date,Current Close,Previous Close (delta=7)
Ticker,,,
SINO,2021-02-11,7.71,8.36
BW,2021-02-11,7.33,6.27
ENLV,2021-02-11,20.00,15.49
MITO,2021-02-11,2.17,1.75
NEA,2021-02-11,14.96,14.98


In [44]:
prices = prices.reset_index()
prices.columns = ['Ticker', 'Date Filed', 'Previous close', 'Next Open']
prices.head()

,Ticker,Date Filed,Previous close,Next Open
0,SINO,2021-02-11,7.40,7.24
1,BW,2021-02-11,7.25,7.75
2,ENLV,2021-02-11,19.18,21.76
3,MITO,2021-02-11,2.35,2.13
4,NEA,2021-02-11,15.03,15.00


In [45]:
train_final = train.merge(prices, on = 'Ticker', how = 'left')
train_final

,CIK Code,Filing,Date Filed_x,Ticker,CompanyName,Date Filed_y,Previous close,Next Open
0,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11,SINO,"Sino-Global Shipping America, Ltd.",2021-02-11,7.400000,7.240000
1,1422892,Beforeyou make a decision to invest in our sec...,2021-02-11,SINO,"Sino-Global Shipping America, Ltd.",2021-02-11,7.400000,7.240000
2,1630805,An investment inthe Notes involves significant...,2021-02-11,BW,"Babcock & Wilcox Enterprises, Inc.",2021-02-11,7.250000,7.750000
3,1630805,An investment inthe Notes involves significant...,2021-02-11,BW,"Babcock & Wilcox Enterprises, Inc.",2021-02-11,7.250000,7.750000
4,1630805,An investment inthe Notes involves significant...,2021-02-11,BW,"Babcock & Wilcox Enterprises, Inc.",2021-02-11,7.250000,7.750000
...,...,...,...,...,...,...,...,...
178,1582581,Investing in our securities involves significa...,2021-02-11,VJET,voxeljet AG,2021-02-11,27.360001,26.280001
179,714256,Investingin our common stock involves a high d...,2021-02-11,SMTI,Sanara MedTech Inc.,2021-02-11,39.000000,27.000000
180,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc,2021-02-11,3.050000,3.200000
181,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc,2021-02-11,3.050000,3.200000


In [46]:
train_final = train_final.drop_duplicates('CIK Code', keep = 'last')
train_final

,CIK Code,Filing,Date Filed_x,Ticker,CompanyName,Date Filed_y,Previous close,Next Open
8,1696396,Investing in our ADSs involves a high degree o...,2021-02-11,MITO,Stealth BioTherapeutics Corp,2021-02-11,2.350000,2.130000
9,1195737,"Investing in the Securities involves risk, inc...",2021-02-11,NEA,Nuveen AMT-Free Quality Municipal Income Fund,2021-02-11,15.030000,15.000000
10,25743,An investment in our common stock involves a h...,2021-02-11,TXMD,"TherapeuticsMD, Inc.",2021-02-11,2.170000,1.990000
16,1730463,Investing in our ADSs involves a high degree o...,2021-02-11,AUTL,Autolus Therapeutics plc,2021-02-11,7.000000,7.300000
17,1579823,"Beforemaking an investment decision, you shoul...",2021-02-11,NBEV,"NewAge, Inc.",2021-02-11,4.370000,3.820000
...,...,...,...,...,...,...,...,...
177,1684144,An investment in our securitiesinvolves a high...,2021-02-11,ZOM,Zomedica Corp.,2021-02-11,2.550000,2.620000
178,1582581,Investing in our securities involves significa...,2021-02-11,VJET,voxeljet AG,2021-02-11,27.360001,26.280001
179,714256,Investingin our common stock involves a high d...,2021-02-11,SMTI,Sanara MedTech Inc.,2021-02-11,39.000000,27.000000
181,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc,2021-02-11,3.050000,3.200000


In [49]:
train_final.drop(columns = 'Date Filed_y', inplace = True)
train_final

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,CIK Code,Filing,Date Filed_x,Ticker,CompanyName,Previous close,Next Open
8,1696396,Investing in our ADSs involves a high degree o...,2021-02-11,MITO,Stealth BioTherapeutics Corp,2.350000,2.130000
9,1195737,"Investing in the Securities involves risk, inc...",2021-02-11,NEA,Nuveen AMT-Free Quality Municipal Income Fund,15.030000,15.000000
10,25743,An investment in our common stock involves a h...,2021-02-11,TXMD,"TherapeuticsMD, Inc.",2.170000,1.990000
16,1730463,Investing in our ADSs involves a high degree o...,2021-02-11,AUTL,Autolus Therapeutics plc,7.000000,7.300000
17,1579823,"Beforemaking an investment decision, you shoul...",2021-02-11,NBEV,"NewAge, Inc.",4.370000,3.820000
...,...,...,...,...,...,...,...
177,1684144,An investment in our securitiesinvolves a high...,2021-02-11,ZOM,Zomedica Corp.,2.550000,2.620000
178,1582581,Investing in our securities involves significa...,2021-02-11,VJET,voxeljet AG,27.360001,26.280001
179,714256,Investingin our common stock involves a high d...,2021-02-11,SMTI,Sanara MedTech Inc.,39.000000,27.000000
181,1719714,"Our business, and an investment in the securit...",2021-02-11,MREO,Mereo Biopharma Group plc,3.050000,3.200000


In [50]:
train_final.to_csv('prelim_train_data', encoding = 'utf-8')